# Universal Robots - Remote Control (URX) Crash Course
v0.3 (2018-03-13)

Welcome to Jupyter Notebook's interactive computational environment.
<ul>
<li>Use ***Shift+Enter*** to **execute cell** *below*</li>
</ul>


In [1]:
%%capture 
import urx #The UR remote control library.
import math3d as m3d #the library, used by urx to represent transformations
from futek import Futek

robot = urx.Robot("172.31.1.25", use_rt=True)

In [4]:
robot

Robot Object (IP=172.31.1.25, state={'size': 47, 'type': 0, 'timestamp': 2524253000, 'isPhysicalRobotConnected': True, 'isRealRobotEnabled': True, 'isPowerOnRobot': True, 'isEmergencyStopped': False, 'isSecurityStopped': False, 'isProgramRunning': False, 'isProgramPaused': True, 'robotMode': 7, 'controlMode': 0, 'speedFraction': 1.0, 'speedScaling': 0.0, 'speedFractionLimit': 63})

## **Hello Robot!**
<ul>
<li>Insert **`New Cell`** (Press key **`B`** to create an empty cell **`B`**elow the selected one)</li>
<li>Press **`Enter`** to switch into *Edit mode*</li>
<li>Type ***`robot.`*** and press **`Tab`** (autocomplete) - you will see all available URX commands</li>
<li>Choose *set_digital_out*</li>
<li>Place *Mouse cursor* on *robot.set_digital_out* and press **`Shift+Tab`** to get *function description*.</li>
<li>This function expects 2 parameters - *digital_output_port_number* and *value* (True/False).</li>
<li>Let's call ***robot.set_digital_out(0,True)***.</li>
<li>Press **`Shift+Enter`** to execute.</li>
</ul>






Take a URobot's teach pendant and open *PolyScope I/O tab* and Check for *digital_output_0* indicator.

In [9]:
robot.set_digital_out(0,True)

In [10]:
robot.set_digital_out(0,False)

In [11]:
# Let's get robot's tool tip position (x, y, z)
# robot.get_pos()
robot.secmon.send_program("force_mode(p[0,0,-1,0,0], [0,0,1,0,0,0], [0,0,40,0,0,0], 2, [.2,.1,.1,.785,.785,1.57])")
# robot.down(z=0.02, acc=0.1, vel=0.1)
p = robot.getl()
p[2] += 0.1
kek = "movel(" + str(p) + ", a=0.01, v=0.1)"
robot.secmon.send_program(kek)

In [15]:
# Let's move robot for 20mm along TOOL Z axis 
# (BE CAREFUL, This command will MOVE the Robot)
robot.up(z=0.2, acc=0.1, vel=0.1)

RobotException: Robot stopped

In [ ]:
# Let's list all URX functions !
help(robot)

Most of the commands are self-descriptive,<br>
But there are three command sub-sets, what will require a deep understanding - Coordinate system, Movement and Force<br><br>

# Robot coordinate system introduction:

<img src="images/coord_sys.png">

### 1. Joints angles 
( j0, j1, j2, j3, j4, j5 ) in Radians from Base_Joint(0) to Whirst3_Joint(5)

In [19]:
robot.getj()                 # get joints angles in RADIANS

[0.07186460494995117,
 -0.562424973850586,
 -0.9837160110473633,
 -1.3876140874675293,
 -0.0008958021747034195,
 -0.22648173967470342]

### 2. Classic Transform
which includes ***Position*** *Vector* from base coordinate sytem to TCP (blue arrow)<br>
and ***Rotation*** *Matrix* https://en.wikipedia.org/wiki/Rotation_matrix

In [30]:
robot.get_pose()             # get transform from current coord system to tcp  

<Transform:
<Orientation: 
array([[ 0.20503014, -0.97869534,  0.01086569],
       [-0.97653575, -0.20380523,  0.06957982],
       [-0.06588296, -0.0248767 , -0.99751721]])>
<Vector: (-0.45263, 0.00755, 0.68464)>
>

In [20]:
robot.get_pos()              # get tool position (x, y, z) in current coord system

<Vector: (-0.45262, 0.00754, 0.68465)>

In [21]:
robot.get_orientation()      # get tool orientation (Rotation Matrix) in current coordinate system

<Orientation: 
array([[ 0.20511099, -0.97867861,  0.0108476 ],
       [-0.97651998, -0.20388759,  0.06955997],
       [-0.06586516, -0.02486042, -0.99751879]])>

### 3. Axis–angle
(x, y, z, Rx, Ry, Rz) from tcp to current coordinate system<br>
x, y, z - stands for Position-Vector - same as get_pos,<br>
Rx, Ry, Rz - is a rotation vector or an axis of rotation of original coord system to get a new one,<br> 
and the lenght of this vector - is an angle of rotation in Radians<br>
https://en.wikipedia.org/wiki/Axis-angle_representation

In [22]:
robot.getl()                 # return current pose (x, y, z, Rx, Ry, Rz) from tcp to current csys 

[-0.45261772594145266,
 0.007543384666263829,
 0.6846517494727873,
 -2.3906589189832648,
 1.9423142310542794,
 0.054655006951792644]

### 4. XYZ

In [3]:
robot.x  # returns current x coordinate

-0.4526303685500654

In [ ]:
robot.y

In [ ]:
robot.z

### 5. Coordinates Conversion
Theory and math of relation between different types of coordinates:<br>
Joints and Pose could be converted using forward and inverse kinematics - http://cdn.intechweb.org/pdfs/379.pdf<br>
RotationMatrix and Axis-Angle - http://www.euclideanspace.com/maths/geometry/rotations/conversions/angleToMatrix/index.htm<br>

In [ ]:
# URX (math3d) library way from axis-angle to transform
axis_angle = robot.getl()
transform = m3d.Transform(axis_angle)
transform

In [ ]:
# and vice-verse:
transform.pose_vector

### 6. Coordinates calculation
Let's say you have a fixture and it is leaned by 30 degrees to horizont. furthermore, angle can be changed <br>
You have a pose inside this fixture and you want to calculate approach pose. approach trajectory should folow pose vector<br> 
Since fixture is leaned you cannot just add a Z coord. You have to multiply matrices:

In [ ]:
def calc_tool_translate(pose, vect):
    p = m3d.Transform(pose)
    t = m3d.Transform()
    t.pos += vect
    new_transform = p * t
    return new_transform.pose_vector

calc_tool_translate(robot.getl(), [0,0,0.1])

<br>

# Movement 

### 1. Simple Move functions 

In [31]:
# Move up along csys z-axis
robot.up(z=0.05, acc=0.01, vel=0.01)

RobotException: Robot stopped

In [32]:
# Move down along csys z-axis
robot.down(z=0.5, acc=0.01, vel=0.01)

RobotException: Robot stopped

In [24]:
# move along tool z-axis
robot.back(z=0.05, acc=0.01, vel=0.01)      

RobotException: Robot stopped

In [ ]:
# Move robot in current coord sys without changing orientation  
robot.translate((0.05, 0, 0.03), acc=0.05, vel=0.05)

In [ ]:
# Move robot in TOOL coord sys without changing orientation
robot.translate_tool((0, 0, 0.03), acc=0.05, vel=0.05)

### 2. Joint space

In [32]:
joints = robot.getj() # get current joints angles  
joints[5] -= 0.5 # add 0.5 radian (30 degrees) to the last joint (wirst3)
robot.movej(joints, acc=0.2, vel=0.2)

RobotException: Robot stopped

### 3. Transformation

In [ ]:
# get current pose, transform it and move robot to new pose
trans = robot.get_pose()  # get current transformation matrix (tool to base)
trans.pos.z += 0.3
trans.orient.rotate_yb(pi/2)
robot.set_pose(trans, acc=0.5, vel=0.2)  # apply the new pose

In [ ]:
# or only work with orientation part
o = robot.get_orientation()
o.rotate_yb(np.pi)
robot.set_orientation(o)

In [ ]:
# Add transform expressed in base coordinate or The same in tool coordinate sys - *add_pose_tool*
# This function will move to new_pose = trans * get_pose()
robot.add_pose_base(trans, acc=0.01, vel=0.01)

### 4. Axis-Angle

These functions are native UR-script commands
###  - Linear Movement

In [39]:
pose = robot.getl()
pose[2] += 0.03
robot.movel(pose, acc=0.01, vel=0.01, wait=True, relative=False, threshold=None)

RobotException: Robot stopped

In [35]:
# Linear Movement in Relative to current pose coordinates
robot.movel((0.1, 0, 0, 0, 0, 0), a, v, relative=true)

NameError: name 'a' is not defined

In [ ]:
# Linear Movement in TCP coordinates
robot.movel_tool([0,0,-0.03,0,0,0.5], acc=0.1, vel=0.1)   

### - Circular movement

In [40]:
# pose_via = robot.getl()
pose_via[0] += 0.05
pose_via[1] += 0.05

pose_to = robot.getl()
pose_to[1] += 0.1

robot.movec(pose_via, pose_to, acc=0.01, vel=0.01, wait=True, threshold=None)

NameError: name 'pose_via' is not defined

### 5. Continious Trajectory

In [47]:
# Let's execute a series of movements (along Z axis in Base coord sys)  
# URX will monitor whether the motion is completed before executing the next one. (URX move functions is blocking)
# Every new command from URX to robot make the robot stop
robot.up(z=0.01, acc=0.1, vel=0.01)
robot.up(z=0.01, acc=0.1, vel=0.01)
robot.up(z=0.01, acc=0.1, vel=0.01)

RobotException: Robot stopped

In [ ]:
# To execute smooth trajectory - all coordinates should be gathered in The List [] and passed via movels() 
p1 = robot.getl()
p1[2] -= 0.01

p2 = robot.getl()
p2[2] -= 0.02

p3 = robot.getl()
p3[2] -= 0.03

points = [p1,p2,p3]

In [ ]:
robot.movels(points, 0.1, 0.1, 0.05)

If you need a constant velocity (moveP instead of moveL) - take a look at <br>
movexs(command, pose_list, acc=0.01, vel=0.01, radius=0.01, wait=True, threshold=None)<br>
where command could be 'movel'/'movep'/'movec'<br>
<br>
If you need to combine a couple types of movement in one smooth trajectory - define a function of those points and movements in UR script language and send it to socket as ASCII string, then execute that function:<br>
s.send(("def movex():\n" + "movel(pose,acc,vel)\n " + "movep(pose,acc,vel)\n " + "movec(pose_via,pose_to,acc,vel)\n " + "end\n").encode())<br>
s.send(("movex()").encode())<br>

or define your own function like movex, but with additional moveC/moveP support.

<br>

## Features and coordinate systems

<img src="images/frames.gif">

In [ ]:
# let's take a look on our current coord-system and save it into back-up variable
base_coord_sys = robot.csys
base_coord_sys

In [ ]:
# Let's switch to the current TCP(Tool center point) coordinate system:
robot.set_csys(robot.get_pose())
robot.csys

In [ ]:
# Let's move the Robot for 20mm along Z axis in NEW COORDINATE SYSTEM:
robot.translate((0, 0, 0.02), acc=0.05, vel=0.01)

In [ ]:
# Let's define a function to create a new coordinate system by three given points
# This is a way how URX and Math3d doing it: 
def get_new_coord_sys(X,Y,Origin):
    new_csys = m3d.Transform.new_from_xyp(X - Origin, Y - Origin, Origin)
    return new_csys

X = m3d.Vector(1,0,0)
Y = m3d.Vector(0,1,0)
Origin = m3d.Vector(0,0,0)
new_coord_sys = get_new_coord_sys(X,Y,Origin)
robot.set_csys(new_coord_sys)
new_coord_sys

In [ ]:
# Let's rotate it to 45 degrees and translate 0.5m along Z axis
X = m3d.Vector(1,1,0)
Y = m3d.Vector(-1,1,0)
Origin = m3d.Vector(0,0,0.5)
new_coord_sys = get_new_coord_sys(X,Y,Origin)
robot.set_csys(new_coord_sys)
new_coord_sys

In [ ]:
# Finnaly, Let's Teach a new coordinate system
# This function will guide you step-by-step through 3 points in teaching mode: X, origin, Y.
# Just RUN it!
new_coord_sys = robot.new_csys_from_xpy()

In [ ]:
# Apply new coordinate system:
robot.set_csys(new_coord_sys)
robot.csys

In [ ]:
# Let's move robot for 20mm along Z axis in NEW COORDINATE SYSTEM !
robot.translate((0, 0, 0.02), acc=0.05, vel=0.01)

In [ ]:
# Return to base coordinate system:
robot.set_csys(base_coord_sys)
robot.csys

## Reference:
https://www.universal-robots.com/how-tos-and-faqs/how-to/ur-how-tos/overview-of-client-interfaces-21744/<br>
https://www.universal-robots.com/how-tos-and-faqs/how-to/ur-how-tos/remote-control-via-tcpip-16496/ - RTDE protocol  discription with source code and examples<br>
https://github.com/jkur/python-urx/tree/SW3.5/urx - in Code Veritas! <br>


In [24]:
robot.stop()


In [2]:
robot.up(z=0.5, acc=0.01, vel=0.01)

Robot not running: {'size': 47, 'type': 0, 'timestamp': 2909107000, 'isPhysicalRobotConnected': True, 'isRealRobotEnabled': True, 'isPowerOnRobot': True, 'isEmergencyStopped': True, 'isSecurityStopped': False, 'isProgramRunning': False, 'isProgramPaused': True, 'robotMode': 4, 'controlMode': 0, 'speedFraction': 1.0, 'speedScaling': 0.0, 'speedFractionLimit': 63}


In [4]:
sensor = Futek(debug=0)
print(sensor.readData())
print("kek")

pose = robot.getl()
pose[2] -= 0.5
# pose[3] = -pose[3]
# pose[4] = -pose[4]
print(pose)
robot.movel(pose, acc=0.01, vel=0.01, wait=False, relative=False, threshold=None)
while True:
    try:
        if sensor.readData(write_to_file=0)>98:
            print("i am here")
            sensor.readData(write_to_file=1)
            robot.stop()
            robot.up(z=0.05, acc=0.01, vel=0.01)
            break
    except Exception:
        print("Bleat")
        pass
print("hooray")
    


Port was opened before
name is futek_data/experiment_kekman.txt
16
kek
[-0.4244087167348145, 0.022987076645380945, -0.5925070865696496, -2.373355516397352, 1.9410139976718213, 0.03165706564231662]
i am here
hooray


In [15]:
robot.down(z=0.5, acc=0.01, vel=0.01)

In [6]:
pose = robot.getl()
pose[2] -= 0.5
# pose[3] = -pose[3]
# pose[4] = -pose[4]
print(pose)
robot.movel(pose, acc=0.01, vel=0.01, wait=False, relative=False, threshold=None)

[-0.4444472441232862, 0.012121259585772318, -0.2893702485559265, -2.3905683897796677, 1.9473462146147962, 0.033794643895700145]


In [3]:
robot.getl()

[-0.4444534123562864,
 0.01212257614589921,
 0.30112728320096727,
 -2.364834305148542,
 1.9262936019366568,
 0.05245509523340704]